# Automate Genuine Chrome

Automate your everyday Chrome browser so your logins are available.

In [ ]:
import asyncio
import getpass
import logging
import platform
import nest_asyncio
from pathlib import Path
from playwright.async_api import async_playwright

# You may wish to change this from default. This script gives you the time & instructions to do so.
user_profile_name = "Default"

# Apply nest_asyncio to run in Jupyter
nest_asyncio.apply()

# Set up logging to capture and silence errors
logging.basicConfig(level=logging.INFO)
logging.getLogger('asyncio').setLevel(logging.CRITICAL)

# User data folder configuration
username = getpass.getuser()
system = platform.system()

# Common paths
windows_base_path = f"C:\\Users\\{username}\\AppData\\Local\\Google\\Chrome\\User Data"
macos_base_path = f"/Users/{username}/Library/Application Support/Google/Chrome"
linux_base_path = f"/home/{username}/.config/google-chrome"

if system == "Windows":
    chrome_path = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"
    user_data_dir = f"{windows_base_path}\\{user_profile_name}"
    profile_dir = windows_base_path
elif system == "Darwin":  # macOS
    chrome_path = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"
    user_data_dir = f"{macos_base_path}/{user_profile_name}"
    profile_dir = macos_base_path
elif system == "Linux":  # Ubuntu/Debian
    chrome_path = "/usr/bin/google-chrome"
    user_data_dir = f"{linux_base_path}/{user_profile_name}"
    profile_dir = linux_base_path
else:
    raise Exception("Unsupported operating system")

def print_instructions():
    print("- This opens Chrome with your Default user profile.")
    print("- If you wish to use a different user profile for automation, make one now.")
    print("- After you close the browser, you can find your new profile here:")
    print()
    print(f"{system}: {profile_dir}")
    print()
    print("Stop this script from running when you're done.")

async def open_browser():
    print_instructions()
    
    async with async_playwright() as plwt:
        context = await plwt.chromium.launch_persistent_context(
            user_data_dir,
            executable_path=chrome_path,
            headless=False,
            accept_downloads=True,
            ignore_default_args=["--enable-automation"]
        )
        
        page = await context.new_page()
        await page.goto("https://photos.google.com/")
        
        try:
            # Keep the browser open until the user closes it, with a very long timeout
            await context.wait_for_event("close", timeout=24*60*60*1000)  # 24 hours
        except asyncio.TimeoutError:
            logging.error("Timeout while waiting for the browser to close.")
        except Exception as e:
            logging.error(f"Unexpected error: {e}")
        finally:
            print(f"\nYour new browser profile can be found in the following directory:")
            print(f"{profile_dir}")

asyncio.run(open_browser())